In [ ]:
import os
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from cmocean import cm
from cartopy.crs import LambertAzimuthalEqualArea
from cartopy.feature import LAND

from pathos.multiprocessing import Pool
%matplotlib inline

In [ ]:
force = False
zoom = False
srs_dst = LambertAzimuthalEqualArea(central_longitude=0, central_latitude=90)
date_start = datetime(1991, 9, 15)
date_end = datetime(1992, 9, 16)

sic_files = []
msh_files = []
titles = []
date0 = date_start
date1 = date_start
while date1 < date_end:
    myi_file = f'../NERSC_arctic25km_sea_ice_age_v2p1/sic/{date0.strftime("%Y")}/sic_{date0.strftime("%Y%m%d")}_{date1.strftime("%Y%m%d")}.npz'
    msh_file = f'../NERSC_arctic25km_sea_ice_age_v2p1/mesh/{date1.strftime("%Y")}/mesh_{date1.strftime("%Y%m%d")}.npz'
    sic_files.append(myi_file)
    msh_files.append(msh_file)
    titles.append(f'$C_{{A1}}$ {date0.strftime("%Y-%m-%d")} --> {date1.strftime("%Y-%m-%d")}')
    date1 += timedelta(1)

frame_counts = list(range(len(sic_files)))
print(len(sic_files))

In [ ]:
class LoadAndPlot:
    def __init__(self, zoom=False, force=False):
        self.zoom = zoom
        self.force = force

    def plot_sic(self, x, y, t, c, title, ofile):
        fig = plt.figure(figsize=(4, 5))
        ax = fig.add_subplot(1, 1, 1, projection=srs_dst)
        ax.add_feature(LAND, facecolor='lightgray', edgecolor='black', zorder=1)
        if zoom == False:
            tcf = ax.tripcolor(x, y, t, c, cmap=cm.ice, zorder=0, clim=[0, 100])
            ax.set_extent([-1700000, 2000000, -2000000, 2200000], crs=srs_dst)
        else:
            tcf = ax.tripcolor(x, y, t, c, cmap=cm.ice, zorder=0, clim=[0, 100], edgecolor='k')
            ax.set_extent([-800000, 800000, -2000000, 0], crs=srs_dst)
        plt.title(title, fontsize=12)
        plt.tight_layout()
        plt.savefig(ofile, dpi=300, bbox_inches='tight', pad_inches=0.1)
        plt.close()

    def __call__(self, input_data):
        frame_counter, sic_file, msh_file, title = input_data
        ofile = f'advect_myi_pngs/frame_{frame_counter:04d}.png'
        if self.zoom == True:
            ofile = ofile.replace('.png', '_zoom.png')
        if os.path.exists(ofile) and not self.force:
            return
        c = np.load(sic_file)['c']
        with np.load(msh_file) as ds:
            x = ds['x']*1000
            y = ds['y']*1000
            t = ds['t']
        self.plot_sic(x, y, t, c, title, ofile)


In [ ]:
#zoom = False
#with Pool(5) as p:
#    p.map(load_and_plot, zip(frame_counts, sic_files, msh_files, titles))

In [ ]:
load_and_plot = LoadAndPlot(zoom=True, force=True)
with Pool(5) as p:
    p.map(load_and_plot, zip(frame_counts, sic_files, msh_files, titles))

In [ ]:
import glob
from PIL import Image

# Get all PNG files in order
png_files = sorted(glob.glob('advect_myi_pngs/frame_????_zoom.png'))

# Load images
images = [Image.open(f) for f in png_files]

# Save as animated GIF
images[0].save('advect_myi_pngs/advect_myi_zoom.gif', save_all=True, append_images=images[1:], duration=100, loop=0)